## Básico

In [50]:
import psycopg2
import requests
import random
import pandas as pd
from faker import Faker
import threading
import csv
from datetime import date
from workalendar.america import Brazil
from datetime import datetime, timedelta

In [48]:
# Função para estabelecer a conexão com o banco de dados PostgreSQL
def conectar_bd():
    try:
        conexao = psycopg2.connect(
            host="localhost",
            port="5432",
            database="sistema_presenca",
            user="postgres",
            password="admin"
        )
        return conexao
    except psycopg2.Error as e:
        print("Erro ao conectar ao banco de dados:", e)
        return None

## Instituições (tb_instituicao)

In [ ]:
# Parâmetros de conexão com o banco de dados PostgreSQL
conexao = conectar_bd()

# Cria um cursor para executar comandos SQL
cursor = conexao.cursor()

# Gera e insere 10 instituições na tabela
for i in range(1, 11):
    nome_instituicao = f"Instituição {i}"
    cursor.execute("INSERT INTO tb_instituicao (co_instituicao, no_instituicao) VALUES (%s, %s)", (i, nome_instituicao))

# Confirma as alterações no banco de dados
conexao.commit()

# Fecha o cursor e a conexão
cursor.close()
conexao.close()

## Professores (tb_professor)

In [44]:
# Crie um objeto Faker configurado para o Português do Brasil (pt_BR)
fake = Faker('pt_BR')

# Abra o arquivo CSV para escrever os dados
with open('professores.csv', mode='w', newline='') as arquivo_csv:
    writer = csv.writer(arquivo_csv, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # Escreva o cabeçalho
    writer.writerow(['co_professor', 'no_professor'])

    # Gere e escreva os dados aleatórios
    for i in range(25000):
        no_professor = fake.name()
        # Remova caracteres especiais da string gerada
        no_professor = no_professor.replace('ã', 'a').replace('õ', 'o').replace('ç', 'c').replace('í', 'i').replace('ó', 'o').replace('á', 'a').replace('é','e').replace('ú','u').replace('Dr. ', '').replace('Dra. ', '').replace('Sr. ','').replace('Sra. ','').replace('Srta. ','')
        writer.writerow([i, no_professor])

print("Arquivo CSV gerado com sucesso!")

Arquivo CSV gerado com sucesso!


In [45]:
# Função para inserir dados em uma tabela
def inserir_dados(cursor, dados):
    inserir_professor = """
    INSERT INTO tb_professor (co_professor, no_professor)
    VALUES (%s, %s)
    """
    cursor.execute(inserir_professor, dados)

# Função para processar um lote de linhas do CSV e inserir na tabela
def processar_lote(cursor, linhas):
    for linha in linhas:
        co_professor, no_professor = linha
        inserir_dados(cursor, (co_professor, no_professor))

# Parâmetros de conexão com o banco de dados PostgreSQL
conexao = conectar_bd()

# Abra o arquivo CSV para leitura
with open('professores.csv', mode='r', newline='') as arquivo_csv:
    reader = csv.reader(arquivo_csv, delimiter=',', quotechar='"')

    # 
    next(reader)
    cursor = conexao.cursor()

    # Defina o número de threads que você deseja usar
    num_threads = 4

    # Leia todas as linhas do CSV e coloque-as em uma lista
    linhas = list(reader)

    # Divida as linhas em lotes iguais para processamento paralelo
    tamanho_lote = len(linhas) // num_threads
    linhas_em_lotes = [linhas[i:i + tamanho_lote] for i in range(0, len(linhas), tamanho_lote)]

    # Crie e inicie threads para processar os lotes
    threads = []
    for lote in linhas_em_lotes:
        t = threading.Thread(target=processar_lote, args=(cursor, lote))
        threads.append(t)
        t.start()

    # Aguarde a conclusão de todas as threads
    for t in threads:
        t.join()

    # Faça o commit das inserções e encerre a transação
    conexao.commit()

    print("Inserções concluídas com sucesso!")

# Feche o cursor e a conexão com o banco de dados
cursor.close()
conexao.close()

Inserções concluídas com sucesso!


## Alunos (tb_aluno)

In [7]:
# Crie um objeto Faker configurado para o Português do Brasil (pt_BR)
fake = Faker('pt_BR')

# Abra o arquivo CSV para escrever os dados
with open('nomes.csv', mode='w', newline='') as arquivo_csv:
    writer = csv.writer(arquivo_csv, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # Escreva o cabeçalho
    writer.writerow(['co_aluno', 'no_aluno', 'dt_nascimento', 'co_instituicao'])

    # Gere e escreva os dados aleatórios
    for i in range(500000):
        no_aluno = fake.name()
        # Remova caracteres especiais da string gerada
        no_aluno = no_aluno.replace('ã', 'a').replace('õ', 'o').replace('ç', 'c').replace('í', 'i').replace('ó', 'o').replace('á', 'a').replace('é','e').replace('ú','u').replace('Dr. ', '').replace('Dra. ', '').replace('Sr. ','').replace('Sra. ','').replace('Srta. ','')
        dt_nascimento = fake.date_of_birth(minimum_age=0, maximum_age=18)
        co_instituicao = random.randint(0, 10)
        writer.writerow([i, no_aluno, dt_nascimento, co_instituicao])

print("Arquivo CSV gerado com sucesso!")

Arquivo CSV gerado com sucesso!


In [8]:
# Função para inserir dados em uma tabela
def inserir_dados(cursor, dados):
    inserir_aluno = """
    INSERT INTO tb_aluno (co_aluno, no_aluno, dt_nascimento, co_instituicao)
    VALUES (%s, %s, %s, %s)
    """
    cursor.execute(inserir_aluno, dados)

# Função para processar um lote de linhas do CSV e inserir na tabela
def processar_lote(cursor, linhas):
    for linha in linhas:
        co_aluno, no_aluno, dt_nascimento, co_instituicao = linha
        inserir_dados(cursor, (co_aluno, no_aluno, dt_nascimento, co_instituicao))

conexao = conectar_bd()

# Abra o arquivo CSV para leitura
with open('nomes.csv', mode='r', newline='') as arquivo_csv:
    reader = csv.reader(arquivo_csv, delimiter=',', quotechar='"')

    # 
    next(reader)
    cursor = conexao.cursor()

    # Defina o número de threads que você deseja usar
    num_threads = 4

    # Leia todas as linhas do CSV e coloque-as em uma lista
    linhas = list(reader)

    # Divida as linhas em lotes iguais para processamento paralelo
    tamanho_lote = len(linhas) // num_threads
    linhas_em_lotes = [linhas[i:i + tamanho_lote] for i in range(0, len(linhas), tamanho_lote)]

    # Crie e inicie threads para processar os lotes
    threads = []
    for lote in linhas_em_lotes:
        t = threading.Thread(target=processar_lote, args=(cursor, lote))
        threads.append(t)
        t.start()

    # Aguarde a conclusão de todas as threads
    for t in threads:
        t.join()

    # Faça o commit das inserções e encerre a transação
    conexao.commit()

    print("Inserções concluídas com sucesso!")

# Feche o cursor e a conexão com o banco de dados
cursor.close()
conexao.close()

Inserções concluídas com sucesso!


## Turnos (tb_turno)

In [9]:
# Parâmetros de conexão com o banco de dados PostgreSQL
conexao = psycopg2.connect(
    host = "localhost",
    port = "5432",
    database = "sistema_presenca",
    user = "postgres",
    password = "admin"
)

# Cria um cursor para executar comandos SQL
cursor = conexao.cursor()

cursor.execute("INSERT INTO tb_turno (co_turno, no_turno) VALUES (%s, %s)", (1, 'Matutino'))
cursor.execute("INSERT INTO tb_turno (co_turno, no_turno) VALUES (%s, %s)", (2, 'Vespertino'))
cursor.execute("INSERT INTO tb_turno (co_turno, no_turno) VALUES (%s, %s)", (3, 'Noturno'))
cursor.execute("INSERT INTO tb_turno (co_turno, no_turno) VALUES (%s, %s)", (4, 'Integral'))

# Confirma as alterações no banco de dados
conexao.commit()

# Fecha o cursor e a conexão
cursor.close()
conexao.close()

## Disciplinas (tb_disciplina)

In [11]:
conexao = conectar_bd()

# Cria um cursor para executar comandos SQL
cursor = conexao.cursor()

cursor.execute("INSERT INTO tb_disciplina (co_disciplina, no_disciplina) VALUES (%s, %s)", (1, 'Língua Portuguesa'))
cursor.execute("INSERT INTO tb_disciplina (co_disciplina, no_disciplina) VALUES (%s, %s)", (2, 'Arte'))
cursor.execute("INSERT INTO tb_disciplina (co_disciplina, no_disciplina) VALUES (%s, %s)", (3, 'Educação Física'))
cursor.execute("INSERT INTO tb_disciplina (co_disciplina, no_disciplina) VALUES (%s, %s)", (4, 'Língua Inglesa'))
cursor.execute("INSERT INTO tb_disciplina (co_disciplina, no_disciplina) VALUES (%s, %s)", (5, 'Matemática'))
cursor.execute("INSERT INTO tb_disciplina (co_disciplina, no_disciplina) VALUES (%s, %s)", (6, 'Ciências'))
cursor.execute("INSERT INTO tb_disciplina (co_disciplina, no_disciplina) VALUES (%s, %s)", (7, 'Geografia'))
cursor.execute("INSERT INTO tb_disciplina (co_disciplina, no_disciplina) VALUES (%s, %s)", (8, 'História'))
cursor.execute("INSERT INTO tb_disciplina (co_disciplina, no_disciplina) VALUES (%s, %s)", (9, 'Ensino Religioso'))
cursor.execute("INSERT INTO tb_disciplina (co_disciplina, no_disciplina) VALUES (%s, %s)", (10, 'Ciências da Natureza e suas Tecnologias'))
cursor.execute("INSERT INTO tb_disciplina (co_disciplina, no_disciplina) VALUES (%s, %s)", (11, 'Ciências Humanas e Sociais Aplicadas'))


# Confirma as alterações no banco de dados
conexao.commit()

# Fecha o cursor e a conexão
cursor.close()
conexao.close()

## Feriados (tb_feriado)

In [36]:
conexao = conectar_bd()

# Crie um cursor para executar comandos SQL
cursor = conexao.cursor()

# Instancie o calendário brasileiro
calendario = Brazil()

# Obtenha uma lista de feriados nacionais para um período de anos (por exemplo, 10 anos)
ano_inicial = date.today().year - 10
ano_final = ano_inicial + 11  # Você pode ajustar o período conforme necessário

contador = 0  # Variável de contagem para substituir o ano

for ano in range(ano_inicial, ano_final):
    feriados = calendario.holidays(ano)

    # Insira os feriados na tabela
    for feriado in feriados:
        no_feriado = feriado[1]
        dt_feriado = feriado[0]
        # Supondo que tp_ambito_feriado, co_municipio, co_uf, tp_remarcado, co_feriado_original, ds_decreto_mudanca
        # sejam valores padrão ou nulos para feriados nacionais
        cursor.execute("INSERT INTO tb_feriado (co_feriado, no_feriado, dt_feriado, tp_ambito_feriado, co_municipio, co_uf, tp_remarcado, co_feriado_original, ds_decreto_mudanca) VALUES (%s, %s, %s, NULL, NULL, NULL, NULL, NULL, NULL)",
                       (contador, no_feriado, dt_feriado))
        contador += 1  # Incrementa o contador após cada inserção

# Faça o commit das inserções e encerre a transação
conexao.commit()

print("Inserção de feriados concluída com sucesso!")

# Feche o cursor e a conexão com o banco de dados
cursor.close()
conexao.close()

Inserção de feriados concluída com sucesso!


## Série/Ano (tb_serie_ano)

In [37]:
conexao = conectar_bd()

# Crie um cursor para executar comandos SQL
cursor = conexao.cursor()

# Defina as séries/anos para cada etapa de ensino
series = [
    (1, 'Berçário'), 
    (2, 'Creche'), 
    (3, 'Pré-escola'), 
    (4, '1º ano'), 
    (5, '2º ano'), 
    (6, '3º ano'), 
    (7, '4º ano'), 
    (8, '5º ano'), 
    (9, '6º ano'), 
    (10, '7º ano'), 
    (11, '8º ano'), 
    (12, '9º ano'), 
    (13, '1ª série'), 
    (14, '2ª série'), 
    (15, '3ª série'), 
    (16, '1º ano (Técnico)'), 
    (17, '2º ano (Técnico)'), 
    (18, '3º ano (Técnico)'), 
    (19, '4º ano (Técnico)')
]

# Insira as séries/anos na tabela
for serie in series:
    cursor.execute("INSERT INTO tb_serie_ano (co_serie, no_serie) VALUES (%s, %s)", serie)

# Faça o commit das inserções e encerre a transação
conexao.commit()

print("Inserção das séries/anos concluída com sucesso!")

# Feche o cursor e a conexão com o banco de dados
cursor.close()
conexao.close()

Inserção das séries/anos concluída com sucesso!


## Turmas (tb_turma)

In [58]:
# Função para inserir dados de turma
def inserir_turmas(conexao):
    cursor = conexao.cursor()

    for co_turma in range(1, 100000):
        no_turma = f"Turma {co_turma}"
        co_serie = random.randint(1, 15)  # Suponha que existam 15 séries possíveis
        co_turno = random.randint(1, 4)
        co_professor = random.randint(0, 24999)  # Suponha que existam 25000 professores
        co_instituicao = random.randint(1, 5)  # Suponha que existam 5 instituições
        nu_ano_letivo = random.randint(2013, 2023)  # Ano letivo entre 2013 e 2023
        co_disciplina = random.randint(1, 11)  # Suponha que existam 11 disciplinas possíveis
        ds_local = None
        st_ativa = nu_ano_letivo == 2023  # Ativa apenas em 2023

        data_inicio, data_fim = gerar_datas_letivo(nu_ano_letivo)

        cursor.execute(
            "INSERT INTO tb_turma (co_turma, no_turma, co_serie, co_turno, co_professor, co_instituicao, dt_inicio_turma, dt_termino_turma, ds_local, nu_ano_letivo, co_disciplina, st_ativa) "
            "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
            (co_turma, no_turma, co_serie, co_turno, co_professor, co_instituicao, data_inicio, data_fim, ds_local, nu_ano_letivo, co_disciplina, st_ativa)
        )
    conexao.commit()

def gerar_horario(co_turno):
    # Gere horários condizentes com o turno
    if co_turno == 1:  # Manhã
        ho_inicio = f"{random.randint(7, 8)}:00"
        ho_termino = f"{random.randint(9, 10)}:00"
    elif co_turno == 2:  # Tarde
        ho_inicio = f"{random.randint(14, 15)}:00"
        ho_termino = f"{random.randint(16, 17)}:00"
    elif co_turno == 3:  # Noite
        ho_inicio = f"{random.randint(18, 19)}:00"
        ho_termino = f"{random.randint(20, 21)}:00"
    else:  # Qualquer horário
        ho_inicio = f"{random.randint(7, 21)}:00"
        ho_termino = f"{random.randint(8, 22)}:00"

    return ho_inicio, ho_termino

# Função para inserir dados de horário
def inserir_horarios(conexao):
    cursor = conexao.cursor()

    for co_turma in range(1, 11):
        co_turno = random.randint(1, 4)
        ho_inicio, ho_termino = gerar_horario(co_turno)

        dias_semana = [1, 2, 3, 4, 5]  # Suponha que 1 seja segunda-feira, 2 terça-feira, etc.

        for dia_semana in dias_semana:
            cursor.execute(
                "INSERT INTO tb_horario (co_turma, nu_dia_semana, ho_inicio, ho_termino) "
                "VALUES (%s, %s, %s, %s)",
                (co_turma, dia_semana, ho_inicio, ho_termino)
            )

    conexao.commit()

def gerar_datas_letivo(nu_ano_letivo):
    data_inicio = datetime(nu_ano_letivo, 1, 1)  # Início em 1 de janeiro do ano letivo
    data_fim = datetime(nu_ano_letivo, 12, 31)  # Término em 31 de dezembro do ano letivo
    return data_inicio, data_fim

# Função para inserir dados de aluno-turma
def inserir_alunos_turmas(conexao):
    cursor = conexao.cursor()

    for co_aluno in range(1, 500000):
        co_turma = random.randint(1, 100000)

        # 95% de probabilidade do aluno estar ativo na turma
        probabilidade_ativo = [True, False]
        weights = [0.95, 0.05]
        st_ativo = random.choices(probabilidade_ativo, weights=weights)[0]

        cursor.execute(
            "INSERT INTO tb_aluno_turma (co_aluno, co_turma, st_ativo) "
            "VALUES (%s, %s, %s)",
            (co_aluno, co_turma, st_ativo)
        )

    conexao.commit()

# Função principal
def main():
    conexao = conectar_bd()
    if conexao:
        inserir_turmas(conexao)
        inserir_horarios(conexao)
        inserir_alunos_turmas(conexao)
        conexao.close()
        print("Dados inseridos com sucesso!")

if __name__ == "__main__":
    main()


Dados inseridos com sucesso!


## Presença

In [ ]:
conexao = conectar_bd()

# Cria um cursor para executar comandos SQL
cursor = conexao.cursor()

cursor.execute("INSERT INTO tb_presenca (co_status_presenca, no_status_presenca, sg_status_presenca) VALUES (%s, %s, %s)", (0, 'Faltou F', 'F'))
cursor.execute("INSERT INTO tb_presenca (co_status_presenca, no_status_presenca, sg_status_presenca) VALUES (%s, %s, %s)", (1, 'Presente P', 'P'))
cursor.execute("INSERT INTO tb_presenca (co_status_presenca, no_status_presenca, sg_status_presenca) VALUES (%s, %s, %s)", (2, 'Licença médica L', 'L'))
cursor.execute("INSERT INTO tb_presenca (co_status_presenca, no_status_presenca, sg_status_presenca) VALUES (%s, %s, %s)", (3, 'Sem matrícula S', 'S'))
cursor.execute("INSERT INTO tb_presenca (co_status_presenca, no_status_presenca, sg_status_presenca) VALUES (%s, %s, %s)", (4, 'Aula cancelada C', 'C'))
cursor.execute("INSERT INTO tb_presenca (co_status_presenca, no_status_presenca, sg_status_presenca) VALUES (%s, %s, %s)", (5, 'Feriado', 'E'))


# Confirma as alterações no banco de dados
conexao.commit()

# Fecha o cursor e a conexão
cursor.close()
conexao.close()

In [72]:
conexao = conectar_bd()

# Crie um cursor para executar comandos SQL
cursor = conexao.cursor()

# Crie um objeto Faker para gerar dados fictícios
fake = Faker()

# Número total de alunos e turmas
num_alunos = 500000
num_turmas = 1000 

# Data de início e término do período de 10 anos
data_inicio = date(2013, 1, 1)
data_fim = date(2023, 12, 31)

# Gere registros de frequência para cada dia útil ao longo dos 10 anos
delta = timedelta(days=1)
data_atual = data_inicio

while data_atual <= data_fim:
    # Verifique se a data é um dia útil (de segunda a sexta-feira)
    if data_atual.weekday() < 5:
        for turma_id in range(1, num_turmas + 1):
            for aluno_id in range(0, num_alunos):
                # Determine se o aluno está presente com 80% de probabilidade
                presente = random.choices([0, 1], weights=[0.2, 0.8])[0]
                
                # Se o aluno estiver presente, escolha um status de presença aleatório
                if presente:
                    co_sub_justificativa = None
                    status_presenca = 1
                    # Inserir registro de frequência
                    cursor.execute(
                        "INSERT INTO tb_frequencia_aluno (co_turma, co_aluno, dt_frequencia, co_status_presenca, co_sub_justificativa) VALUES (%s, %s, %s, %s, %s)",
                        (turma_id, aluno_id, data_atual, status_presenca, co_sub_justificativa)
                    )
                
                else:
                    co_sub_justificativa = random.randint(1, 82)
                    status_presenca = 0
                    # Inserir registro de frequência
                    cursor.execute(
                        "INSERT INTO tb_frequencia_aluno (co_turma, co_aluno, dt_frequencia, co_status_presenca, co_sub_justificativa) VALUES (%s, %s, %s, %s, %s)",
                        (turma_id, aluno_id, data_atual, status_presenca, co_sub_justificativa)
                    )
    
    # Avance para o próximo dia
    data_atual += delta

# Faça o commit das inserções e encerre a transação
conexao.commit()

# Feche o cursor e a conexão com o banco de dados
cursor.close()
conexao.close()

print("Inserções concluídas com sucesso!")


KeyboardInterrupt: 